In [1]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [2]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director
genre_name_df = pd.read_csv('/opt/ml/input/data/train/genres.tsv', sep='\t') # item-genre(name)
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

In [3]:
main_df

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563
...,...,...,...
5154466,138493,44022,1260209449
5154467,138493,4958,1260209482
5154468,138493,68319,1260209720
5154469,138493,40819,1260209726


In [4]:
# 유저별 좋아하는 감독 top3

In [5]:
no_dir_item_list = list(set(main_df['item']) - set(director_df['item']))
no_dir_item_df = pd.DataFrame([x for x in zip(no_dir_item_list, ['nm0000000']*len(no_dir_item_list))])
no_dir_item_df.columns=director_df.columns # 컬럼명 동일하게
director_df = pd.concat([director_df, no_dir_item_df]) # 기존 director_df 뒤에 감독없는 영화 추가

In [6]:
df = main_df.merge(director_df, how='left', on='item')

In [10]:
df[df['user']==11]['director'].value_counts()

nm0000000    43
nm0000229    10
nm0000318     6
nm0000709     6
nm0000386     6
             ..
nm0942367     1
nm0957772     1
nm0269463     1
nm0812200     1
nm0267512     1
Name: director, Length: 258, dtype: int64

In [15]:
df[df['user']==11]['director'].value_counts().index[:4]

Index(['nm0000000', 'nm0000229', 'nm0000318', 'nm0000709'], dtype='object')

In [17]:
user_list = df['user'].unique()

In [ ]:
#일단 top3명을 뽑는데, 같은 횟수의 감독 -> 인기도기반

In [18]:
result = []
for user in user_list:
    like_director = df[df['user']==user]['director'].value_counts().index[:4]
    tmp = []
    for direct in like_director:
        if direct != 'nm0000000':
            tmp.append(direct)
    first, second, third = tmp[:3]
    result.append((user, first, second, third))
    
    
user_direct3_df = pd.DataFrame(result)

In [19]:
user_direct3_df

,0,1,2,3
0,11,nm0000229,nm0000318,nm0000709
1,14,nm0000229,nm0000709,nm0414144
2,18,nm0000264,nm0001466,nm0600546
3,25,nm0000229,nm0000116,nm0000416
4,31,nm0001392,nm0893659,nm0946734
...,...,...,...,...
31355,138473,nm0594503,nm0000233,nm0000318
31356,138475,nm0000033,nm0001328,nm0000485
31357,138486,nm0000709,nm0001681,nm0000229
31358,138492,nm0001054,nm0001554,nm0001661


In [21]:
user_direct3_df.columns = ['user','director1','director2','director3']

In [22]:
user_direct3_df.to_csv(
    "user_director_fe.csv", index=False
)